In [24]:
import torch
import random
import evaluate

from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer

In [14]:
peft_model_name = 'roberta-base-peft'
modified_base = 'roberta-base-modified'
base_model = 'roberta-base'

In [3]:
dataset = load_dataset('ag_news')
tokenizer = RobertaTokenizer.from_pretrained(base_model)

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
len(dataset['train'])

120000

In [5]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}

In [6]:

def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)
    
    
tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
train_dataset=tokenized_dataset['train']

# divide the total test data into two equal sets using shard
test_dataset=tokenized_dataset['test'].shard(num_shards=2, index=0)
eval_dataset=tokenized_dataset['test'].shard(num_shards=2, index=1)

In [7]:
print(dataset['train'].features['label']) 
# look at the print to find about the right key.
class_labels = dataset['train'].features['label'].names
num_labels = len(class_labels)
print(f"number of labels: {num_labels}")
print(f"the labels: {class_labels}")

ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)
number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [8]:
# Create an (id:label) mapping for clasisfing
id2label = {i: label for i, label in enumerate(class_labels)}
print(id2label)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")
print(data_collator)

{0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}
DataCollatorWithPadding(tokenizer=RobertaTokenizer(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}, padding=True, m

In [9]:
# Trainign arguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
)

def get_trainer(model, training_args, train_dataset):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

c:\Users\incognito\miniconda3\envs\llm\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
# model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)
# fine_tune_trainer = get_trainer(model, training_args, train_dataset)

# fine_tune_trainer.train()

In [11]:
# PEFT Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model_ft = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
peft_model = get_peft_model(model_ft, peft_config)
peft_model.to(device)
print("PEFT MODEL")
peft_model.print_trainable_parameters()
peft_lora_finetuning_trainer = get_trainer(peft_model, training_args, train_dataset)
peft_lora_finetuning_trainer.train()

cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PEFT MODEL
trainable params: 888,580 || all params: 125,537,288 || trainable%: 0.7078


  7%|▋         | 500/7500 [03:28<50:33,  2.31it/s]

{'loss': 0.6053, 'grad_norm': 6.2370758056640625, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.07}


...

                                                   
100%|██████████| 7500/7500 [59:30<00:00,  2.51it/s]

{'eval_loss': 0.22215646505355835, 'eval_runtime': 27.793, 'eval_samples_per_second': 136.725, 'eval_steps_per_second': 17.091, 'epoch': 1.0}


100%|██████████| 7500/7500 [59:30<00:00,  2.10it/s]

{'train_runtime': 3570.8821, 'train_samples_per_second': 33.605, 'train_steps_per_second': 2.1, 'train_loss': 0.2802134836832682, 'epoch': 1.0}


TrainOutput(global_step=7500, training_loss=0.2802134836832682, metrics={'train_runtime': 3570.8821, 'train_samples_per_second': 33.605, 'train_steps_per_second': 2.1, 'total_flos': 2.0500492798385664e+16, 'train_loss': 0.2802134836832682, 'epoch': 1.0})

In [15]:
# Save
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

In [18]:
# Testing(Inference) of the LORA fine tune model.

# peft_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model, id2label=id2label)
tokenizer = AutoTokenizer.from_pretrained(modified_base)

def classify(text):
    inputs = tokenizer(text,truncation=True, padding=True, return_tensors='pt')
    output = peft_model(**input)
    
    prediction = output.logits.argmax(dim=-1).item()
    
    print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
    

In [19]:
classify( "Kederis proclaims innocence Olympic champion Kostas Kederis today left hospital ahead of his date with IOC inquisitors claiming his ...")
classify( "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")

TypeError: PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_

In [34]:
metric = evaluate.load('accuracy')

def evaluate_model(model, dataset):
    
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator).to
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    model.to(device)
    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=1)
        references = batch['labels']
        metric.add_batch(
            predictions=predictions,
            references=references
        )
    
    eval_metric = metric.compute()
    print(eval_metric)

In [35]:
# Base model evaluation on test data
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label), test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# LORA (peft) model evaluation on test data
evaluate_model(peft_model, test_dataset)

In [ ]:
# Fully Fine tuned model evaluation on test data
# evaluate_model(fine_tune_trainer.model, test_dataset)